# Pump Model

The `Pump` class represents a controllable pump in a water system. It can be turned on or off to provide a specific flow rate, provided it can overcome the head difference between the upstream and downstream points. This notebook explains the `Pump` model and demonstrates its use in a simulation.

## State Variables

The state of the pump is described by:

- `status` (int): The operational status of the pump (0 for off, 1 for on).
- `outflow` (float): The flow rate produced by the pump (m^3/s).
- `power_draw_kw` (float): The electrical power consumed by the pump (kW).

## Parameters

The pump's operational characteristics are defined by:

- `max_head` (float): The maximum pressure head the pump can overcome (m).
- `max_flow_rate` (float): The flow rate of the pump when it is on and the head is within limits (m^3/s).
- `power_consumption_kw` (float): The power consumed by the pump when it is running (kW).

## Simulation Example

The following example simulates the operation of a pump. The pump is turned on and off during the simulation, and the upstream and downstream water levels are defined. The simulation tracks the pump's status, outflow, and power consumption, which are then plotted.

**Note:** To run this notebook, ensure you have `matplotlib` installed (`pip install matplotlib`) and that you are running the Jupyter server from the root directory of the project.

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.pump import Pump
from swp.core.interfaces import State, Parameters

# Initial state and parameters for the pump
initial_state = State(status=0, outflow=0.0, power_draw_kw=0.0)
parameters = Parameters(
    max_head=25.0,             # meters
    max_flow_rate=2.0,         # m^3/s
    power_consumption_kw=150.0 # kW
)

# Create a Pump instance
pump = Pump(name="main_pump", initial_state=initial_state, parameters=parameters)

# Simulation settings
dt = 1  # time step in seconds
simulation_duration = 100
num_steps = int(simulation_duration / dt)

# Store results for plotting
history = []

# Define external conditions
upstream_head = 10.0
downstream_head = 30.0

# Run the simulation loop
for t in range(num_steps):
    # Turn the pump on for a period, then off
    if 20 <= t < 80:
        control_signal = 1  # On
    else:
        control_signal = 0  # Off
    
    action = {
        'control_signal': control_signal,
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = pump.step(action=action, dt=dt)
    history.append(current_state.copy())

# Print the final state
print("Final State:", pump.get_state())

# Prepare data for plotting
time_steps = [i * dt for i in range(num_steps)]
statuses = [s['status'] for s in history]
outflows = [s['outflow'] for s in history]
power_draws = [s['power_draw_kw'] for s in history]

# Plot the results
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

ax1.plot(time_steps, statuses, label='Pump Status', drawstyle='steps-post')
ax1.set_ylabel('Status (0=Off, 1=On)')
ax1.set_title('Pump Operation Simulation')
ax1.grid(True)
ax1.legend()

ax2.plot(time_steps, outflows, label='Outflow', color='orange')
ax2.set_ylabel('Outflow (m^3/s)')
ax2.grid(True)
ax2.legend()

ax3.plot(time_steps, power_draws, label='Power Draw', color='green')
ax3.set_xlabel('Time (seconds)')
ax3.set_ylabel('Power (kW)')
ax3.grid(True)
ax3.legend()

plt.tight_layout()
plt.show()
